# Using YOLO 8V to a Object Detection Model.

**Creating a image and video detector model from soccer object**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 8.1 MB/s eta 0:00:00


In [2]:
## importing required libraries
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [3]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [6]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/YOLOv8_Object_det/dataset/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 66 images---


  0%|          | 0/53 [00:00<?, ?it/s]

------ Training data created with 80% split 53 images -------


  0%|          | 0/13 [00:00<?, ?it/s]

------ Testing data created with a total of 13 images ----------
------ TASK COMPLETED -------


In [7]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.203 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.2/78.2 GB disk)


In [11]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/YOLOv8_Object_det/dataset.yaml epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/YOLOv8_Object_det/training_result name=football

Ultralytics YOLOv8.0.203 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/YOLOv8_Object_det/dataset.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/YOLOv8_Object_det/training_result, name=football, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, cl

In [12]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/YOLOv8_Object_det/training_result/football/weights/best.pt conf=0.55 source=/content/drive/MyDrive/YOLOv8_Object_det/test_set

Ultralytics YOLOv8.0.203 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/10 /content/drive/MyDrive/YOLOv8_Object_det/test_set/football-4455306_640.jpg: 448x640 (no detections), 67.9ms
image 2/10 /content/drive/MyDrive/YOLOv8_Object_det/test_set/football-5180297_640.jpg: 640x448 4 players, 64.2ms
image 3/10 /content/drive/MyDrive/YOLOv8_Object_det/test_set/gettyimages-1176778885-2048x2048.jpg: 416x640 23 players, 67.2ms
image 4/10 /content/drive/MyDrive/YOLOv8_Object_det/test_set/gettyimages-1287265345-612x612.jpg: 448x640 (no detections), 13.6ms
image 5/10 /content/drive/MyDrive/YOLOv8_Object_det/test_set/jump-1822412_640.jpg: 448x640 3 players, 12.9ms
image 6/10 /content/drive/MyDrive/YOLOv8_Object_det/test_set/pexels-photo-1198172.jpg: 640x448 4 players, 12.7ms
image 7/10 /content/drive/MyDrive/YOLOv8_Object_det/test_set/pexels-photo-1657328.jpg: 640x448 7 players, 2 footballs, 11

In [13]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/YOLOv8_Object_det/output

In [14]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/YOLOv8_Object_det/training_result/football/weights/best.pt conf=0.77 source=/content/drive/MyDrive/YOLOv8_Object_det/vid1

Streaming output truncated to the last 5000 lines.
video 1/5 (1281/1319) /content/drive/MyDrive/YOLOv8_Object_det/vid1/video1.mp4: 640x384 5 players, 2 footballs, 10.9ms
video 1/5 (1282/1319) /content/drive/MyDrive/YOLOv8_Object_det/vid1/video1.mp4: 640x384 4 players, 1 football, 10.8ms
video 1/5 (1283/1319) /content/drive/MyDrive/YOLOv8_Object_det/vid1/video1.mp4: 640x384 4 players, 1 football, 10.8ms
video 1/5 (1284/1319) /content/drive/MyDrive/YOLOv8_Object_det/vid1/video1.mp4: 640x384 3 players, 1 football, 10.8ms
video 1/5 (1285/1319) /content/drive/MyDrive/YOLOv8_Object_det/vid1/video1.mp4: 640x384 2 players, 10.8ms
video 1/5 (1286/1319) /content/drive/MyDrive/YOLOv8_Object_det/vid1/video1.mp4: 640x384 3 players, 1 football, 10.8ms
video 1/5 (1287/1319) /content/drive/MyDrive/YOLOv8_Object_det/vid1/video1.mp4: 640x384 2 players, 1 football, 10.8ms
video 1/5 (1288/1319) /content/drive/MyDrive/YOLOv8_Object_det/vid1/video1.mp4: 640x384 1 player, 10.7ms
video 1/5 (1289/1319) /conten

In [15]:
!cp -r /content/runs/detect/predict2 /content/drive/MyDrive/YOLOv8_Object_det/output

In [16]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/YOLOv8_Object_det/training_result/football/weights/best.pt conf=0.77 source=/content/drive/MyDrive/YOLOv8_Object_det/vid2

Ultralytics YOLOv8.0.203 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

video 1/4 (1/671) /content/drive/MyDrive/YOLOv8_Object_det/vid2/video6.mp4: 640x384 (no detections), 107.0ms
video 1/4 (2/671) /content/drive/MyDrive/YOLOv8_Object_det/vid2/video6.mp4: 640x384 1 player, 10.8ms
video 1/4 (3/671) /content/drive/MyDrive/YOLOv8_Object_det/vid2/video6.mp4: 640x384 (no detections), 10.9ms
video 1/4 (4/671) /content/drive/MyDrive/YOLOv8_Object_det/vid2/video6.mp4: 640x384 1 player, 10.8ms
video 1/4 (5/671) /content/drive/MyDrive/YOLOv8_Object_det/vid2/video6.mp4: 640x384 1 player, 10.8ms
video 1/4 (6/671) /content/drive/MyDrive/YOLOv8_Object_det/vid2/video6.mp4: 640x384 1 player, 10.9ms
video 1/4 (7/671) /content/drive/MyDrive/YOLOv8_Object_det/vid2/video6.mp4: 640x384 1 player, 10.7ms
video 1/4 (8/671) /content/drive/MyDrive/YOLOv8_Object_det/vid2/video6.mp4: 640x384 2 players, 10.8ms
video 

In [17]:
!cp -r /content/runs/detect/predict3 /content/drive/MyDrive/YOLOv8_Object_det/output


In [20]:
!yolo mode=export model=/content/drive/MyDrive/YOLOv8_Object_det/training_result/football/weights/best.pt format=onnx

Ultralytics YOLOv8.0.203 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/YOLOv8_Object_det/training_result/football/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (21.5 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 1.4s, saved as '/content/drive/MyDrive/YOLOv8_Object_det/training_result/football/weights/best.onnx' (42.6 MB)

Export complete (3.9s)
Results saved to /content/drive/MyDrive/YOLOv8_Object_det/training_result/football/weights
Predict:         yolo predict task=detect model=/content/drive/MyDrive/YOLOv8_Object_det/training_result/football/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=/content/drive/MyDrive/YOLOv8_Object_det/training_result/football/weights/best.onnx imgsz=640 data=/content/drive/MyDrive/YOLOv8_Object_det/dataset.yaml  
